In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
!pip install pytorch-transformers
!pip install transformers
!pip install nltk


     |████████████████████████████████| 184kB 5.7MB/s 
     |████████████████████████████████| 1.2MB 34.3MB/s 
     |████████████████████████████████| 133kB 46.2MB/s 
     |████████████████████████████████| 890kB 16.0MB/s 
     |████████████████████████████████| 7.3MB 41.3MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a3cb3b235216192f83df7944a8ea6ebb3c671b4e91184101cb00bc204f04a529
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.27 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.9MB 6.0MB/s 
     |████████████████████████████████| 3.2MB 36.7MB/s 


In [3]:
!pip install tweet-preprocessor

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLMModel, BertTokenizer, BertForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification
from transformers import AdamW
import nltk
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# % matplotlib inline


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla K80'

In [7]:
MAX_LEN = 128

In [8]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news/COVID Fake News Data.csv')


In [10]:
train.head()

,headlines,outcome
0,A post claims compulsory vacination violates t...,0
1,A photo claims that this person is a doctor wh...,0
2,Post about a video claims that it is a protest...,0
3,All deaths by respiratory failure and pneumoni...,0
4,The dean of the College of Biologists of Euska...,0


In [11]:
train = train.rename(columns={ 'headlines' : 'text'})
train = train.rename(columns={ 'outcome' : 'label'})

In [12]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 6120
Size of validation set: 2040
Size of testing set: 2041


In [13]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()


In [14]:
p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess(row, lemmatizer, stemmer):
    text = row['text']
    # text = text.strip('\xa0')
    text = p.clean(text)
    tokenization = nltk.word_tokenize(text)     
    tokenization = [w for w in tokenization if not w in stop_words]
    #   text = ' '.join([porter_stemmer.stem(w) for w in tokenization])
    #   text = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
    # text = re.sub(r'\([0-9]+\)', '', text).strip()    
    return text


In [15]:
df['text'] = df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
val_df['text'] = val_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
test_df['text'] = test_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)


In [16]:
def map_label(row):
    return 0 if row['label']==1 else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [17]:
train_sentences = df.text.values
val_sentences = val_df.text.values
test_sentences = test_df.text.values

train_labels = df.label_encoded.values
val_labels = val_df.label_encoded.values


In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [19]:
def Encode_TextWithAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=True):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids'],encoded_dict['attention_mask']

def Encode_TextWithoutAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=False):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids']

def get_TokenizedTextWithAttentionMask(sentenceList, tokenizer):
    token_ids_list,attention_mask_list = [],[]
    for sentence in sentenceList:
        token_ids,attention_mask = Encode_TextWithAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
        attention_mask_list.append(attention_mask)
    return token_ids_list,attention_mask_list

def get_TokenizedText(sentenceList, tokenizer):
    token_ids_list = []
    for sentence in sentenceList:
        token_ids = Encode_TextWithoutAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
    return token_ids_list

In [20]:
train_token_ids,train_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(train_sentences,tokenizer))
val_token_ids,val_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(val_sentences,tokenizer))
test_token_ids,test_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(test_sentences,tokenizer))

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [21]:
batch_size = 32

train_data = TensorDataset(train_token_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_token_ids, val_attention_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_token_ids, test_attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [22]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).cuda()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [23]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [24]:
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [25]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [27]:
train_loss_set = []
best_val_accuracy = 0.10
directory_path = '/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news'
epochs = 10

for _ in trange(epochs, desc="Epoch"):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        train_loss_set.append(loss.item())    
        loss.backward()
        optimizer.step()

        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          logits = output[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    Validation_Accuracy = (eval_accuracy/nb_eval_steps)
    if(Validation_Accuracy >= best_val_accuracy):
        torch.save(model.state_dict(), directory_path+'/models/BERT_base_uncased_best_model.ckpt')
        best_val_accuracy = Validation_Accuracy
        print('Model Saved')




Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.14365997889156765
Validation Accuracy: 0.95703125


Epoch:  10%|█         | 1/10 [04:16<38:25, 256.19s/it]

Model Saved
Train loss: 0.06642451771525278
Validation Accuracy: 0.978515625


Epoch:  20%|██        | 2/10 [08:34<34:15, 256.88s/it]

...

Validation Accuracy: 0.978515625
Train loss: 0.003458148857437967


Epoch:  90%|█████████ | 9/10 [38:28<04:15, 255.58s/it]

Validation Accuracy: 0.97900390625
Train loss: 0.00638658731046841
Validation Accuracy: 0.98193359375


Epoch: 100%|██████████| 10/10 [42:45<00:00, 256.51s/it]

Model Saved


In [28]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLNetTokenizer, XLNetForSequenceClassification,BertTokenizer, BertForSequenceClassification, DebertaTokenizer, DebertaForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, ElectraTokenizer, ElectraForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla K80'

In [30]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news/COVID Fake News Data.csv')

In [31]:
train = train.rename(columns={ 'headlines' : 'text'})
train = train.rename(columns={ 'outcome' : 'label'})

In [32]:
train['id'] = range(1, len(train) + 1)

In [33]:
train.head()

,text,label,id
0,A post claims compulsory vacination violates t...,0,1
1,A photo claims that this person is a doctor wh...,0,2
2,Post about a video claims that it is a protest...,0,3
3,All deaths by respiratory failure and pneumoni...,0,4
4,The dean of the College of Biologists of Euska...,0,5


In [34]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
test_label_df = test_df

print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 6120
Size of validation set: 2040
Size of testing set: 2041


In [35]:
def preprocess(row):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [36]:
def map_label(row):
  return 0 if row['label']==1 else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [37]:
train_sentences = df.text.values
train_token_ids = df.id.values
val_sentences = val_df.text.values
val_token_ids = val_df.id.values
test_sentences = test_df.text.values
test_token_ids = test_df.id.values


In [38]:
train_sentences = [sentence + " [SEP] [CLS]" for sentence in train_sentences]
train_labels = df.label_encoded.values
val_sentences = [sentence + " [SEP] [CLS]" for sentence in val_sentences]
val_labels = val_df.label_encoded.values
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]


In [39]:
MAX_LEN = 128
batch_size = 64


In [40]:
def get_dataloader(network, train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids):
  dic = {"XLNET": "xlnet-base-cased", "BERT": "bert-base-uncased", "ROBERT": "roberta-base", "XLM-ROBERT": "xlm-roberta-base", "DeBERTa": "microsoft/deberta-base", "Electra": "google/electra-base-discriminator"}
  if network=='XLNET':
    tokenizer = XLNetTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='DeBERTa':
    tokenizer = DebertaTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='BERT':
    tokenizer = BertTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='Electra':
    tokenizer = ElectraTokenizer.from_pretrained(dic[network], do_lower_case=True)
  else:
    tokenizer = RobertaTokenizerFast.from_pretrained(dic[network], do_lower_case=True)

  tokenized_train_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
  print ("Tokenize the first sentence:")
  print (tokenized_train_texts[0])

  tokenized_val_texts = [tokenizer.tokenize(sent) for sent in val_sentences]
  tokenized_test_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
  input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train_texts]
  input_val_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_val_texts]
  input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test_texts]

  input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_val_ids = pad_sequences(input_val_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Create attention masks
  train_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_train_ids:
    seq_mask = [float(i>0) for i in seq]
    train_attention_masks.append(seq_mask)
  train_masks = np.array(train_attention_masks)

  # Create attention masks
  val_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_val_ids:
    seq_mask = [float(i>0) for i in seq]
    val_attention_masks.append(seq_mask)
  validation_masks = np.array(val_attention_masks)

  # Create attention masks
  test_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_test_ids:
    seq_mask = [float(i>0) for i in seq]
    test_attention_masks.append(seq_mask)
  test_masks = np.array(test_attention_masks)

  train_inputs = torch.tensor(input_train_ids)
  validation_inputs = torch.tensor(input_val_ids)
  test_inputs = torch.tensor(input_test_ids)
  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(val_labels)
  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)
  test_masks = torch.tensor(test_masks)

  train_data = TensorDataset(torch.tensor(train_token_ids), train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(torch.tensor(val_token_ids), validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  test_data = TensorDataset(torch.tensor(test_token_ids), test_inputs, test_masks)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  return train_dataloader, validation_dataloader, test_dataloader


In [41]:
train_dataloader_BERT, val_dataloader_BERT, test_dataloader_BERT = get_dataloader("BERT", train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids)


Tokenize the first sentence:
['burning', 'candles', 'kills', 'corona', '##virus', '?', 'pm', '’', 's', 'message', 'given', 'false', 'spin', '[SEP]', '[CLS]']


In [42]:
model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
directory_path = '//content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news/models'
model2.load_state_dict(torch.load(directory_path+'/BERT_base_uncased_best_model.ckpt'))
model2.eval()
model2.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [43]:
def get_model_preds_labels(model, dataloader, mode='train'):
  data_vectors = []
  labels = []
  ids = []

  with torch.no_grad():
      correct = 0
      total = 0
      for i, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if mode=='test':
            token_ids, b_input_ids, b_input_mask = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            for point, tk_id in zip(F.softmax(outputs.logits).tolist(), token_ids.tolist()):
              data_vectors.append(point)       
              ids.append(tk_id)     
        else:
          # Unpack the inputs from our dataloader
          token_ids, b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          for point, lab, tk_id in zip(F.softmax(outputs.logits).tolist(), b_labels.tolist(), token_ids.tolist()):
            data_vectors.append(point)
            labels.append(lab)
            ids.append(tk_id)

  if mode=='test':
    return data_vectors, ids
  else:
    return data_vectors, labels, ids


#Train Data

In [44]:
m2, _, train_ids2 = get_model_preds_labels(model2, train_dataloader_BERT)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [45]:
pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in m2]
train_pred_df = pd.DataFrame({'id': train_ids2, 'predicted_label': pred_labels})
train_pred_df = df[["id", "label"]].merge(train_pred_df, on='id', how='left')
train_pred_df.head()

,id,label,predicted_label
0,1495,0,0
1,3978,0,0
2,1431,0,0
3,4127,0,0
4,2735,0,0


In [46]:
train_pred_df.shape

(6120, 3)

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score


In [48]:
confusion_matrix(train_pred_df['label'].values, train_pred_df['predicted_label'].values)

array([[5821,   23],
       [  44,  232]])

In [49]:
accuracy_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values)

0.9890522875816994

In [50]:
precision_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro')

0.9890522875816994

In [51]:
recall_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro')

0.9890522875816994

In [52]:
f1_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro')

0.9890522875816994

#Validation Data

In [53]:
v2, _, v_ids2 = get_model_preds_labels(model2, val_dataloader_BERT)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [54]:
val_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in v2]
val_pred_df = pd.DataFrame({'id': v_ids2, 'predicted_label': val_pred_labels})
val_pred_df = val_df[["id", "label"]].merge(val_pred_df, on='id', how='left')
val_pred_df.head()

,id,label,predicted_label
0,1552,0,0
1,5686,0,0
2,8012,0,0
3,9469,0,0
4,6867,0,0


In [55]:
confusion_matrix(val_pred_df['label'].values, val_pred_df['predicted_label'].values)

array([[1926,   14],
       [  17,   83]])

In [56]:
accuracy_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values)

0.9848039215686275

In [57]:
precision_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro')

0.9848039215686275

In [58]:
recall_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro')

0.9848039215686275

In [59]:
f1_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro')

0.9848039215686275

#Test Data

In [60]:
t2, t_ids2 = get_model_preds_labels(model2, test_dataloader_BERT, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [61]:
test_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in t2]
test_pred_df = pd.DataFrame({'id': t_ids2, 'predicted_label': test_pred_labels})
test_pred_df = test_label_df[["id", "label"]].merge(test_pred_df, on='id', how='left')
test_pred_df.head()

,id,label,predicted_label
0,5382,0,0
1,10049,0,0
2,8627,0,0
3,9142,0,0
4,9046,0,0


In [62]:
confusion_matrix(test_pred_df['label'].values, test_pred_df['predicted_label'].values)

array([[1939,    4],
       [  18,   80]])

In [63]:
accuracy_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values)

0.9892209701126898

In [64]:
precision_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro')

0.9892209701126898

In [65]:
recall_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro')

0.9892209701126898

In [66]:
f1_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro')

0.9892209701126898